In [3]:
import pm4py
import pandas as pd

log = pm4py.read_xes("../logs/sepsis.xes")
# log = pd.read_csv("../logs/sepsis_subset.csv")

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

In [4]:
from convert_log import convert_log_rdf, RdfRepresent
from rdflib import Namespace

Sepsis = Namespace("http://dutch.hospital.nl/sepsis#")

# convert_log_rdf(log, "../logs/sepsis_pred_25.n3", RdfRepresent.LINK_PRED, 250)
# convert_log_rdf(log, "../logs/sepsis_50.n3", RdfRepresent.LINK_PRED, 500)
# convert_log_rdf(log, "../logs/sepsis_pred_all.n3", RdfRepresent.LINK_PRED)
convert_log_rdf(log, "../logs/sepsis_subset-temp.n3", Sepsis, RdfRepresent.LINK_PRED)

total number of traces: 1050
conversion time (ms): 1959.975
save time (ms): 1656.868
total time (ms): 3616.997
